In [1]:
import ee, pandas as pd, geemap
ee.Authenticate() # Authenticate Earth Engine
ee.Initialize() # Initialize the API
print("Earth Engine is ready!")
print("EE Version:", ee.__version__) # Print Earth Engine version

Earth Engine is ready!
EE Version: 1.6.14


In [5]:
print("Point coordinates:", point.getInfo()) # Create a simple point

NameError: name 'point' is not defined

## Outcome of the following code
Will get a table like:

* year

* total_images (tiles intersecting the county that year)

* images_with_N (how many include NIR band "N")

* band_sets (e.g., "R,G,B" vs "R,G,B,N")



In [3]:
# AOI: Mecklenburg County, NC
meck = (ee.FeatureCollection('TIGER/2018/Counties')
        .filter(ee.Filter.eq('NAME', 'Mecklenburg'))
        .filter(ee.Filter.eq('STATEFP', '37')))

# NAIP over AOI, add helper properties (year and band list as string)
def add_props(img):
    yr = ee.Date(img.get('system:time_start')).get('year')
    bands_str = img.bandNames().join(',')    # e.g., "R,G,B,N"
    return img.set({'year': yr, 'bands_str': bands_str})

naip_all = (ee.ImageCollection('USDA/NAIP/DOQQ')
            .filterBounds(meck)
            .map(add_props))

# Distinct years present
years = ee.List(naip_all.aggregate_array('year')).distinct().sort()
print('Years available:', years.getInfo())

# Build a small per-year table on the server
def summarize(y):
    icY = naip_all.filter(ee.Filter.eq('year', y))
    total   = icY.size()
    withN   = icY.filter(ee.Filter.listContains('system:band_names','N')).size()
    bandsets = icY.aggregate_array('bands_str').distinct().sort()
    return ee.Feature(None, {
        'year': y,
        'total_images': total,
        'images_with_N': withN,
        'band_sets': bandsets
    })

byYear_fc = ee.FeatureCollection(years.map(summarize))

# Bring to pandas for a clean view
rows = [f['properties'] for f in byYear_fc.getInfo()['features']]
summary_df = pd.DataFrame(rows).sort_values('year')
summary_df


Years available: [2004, 2005, 2006, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


,band_sets,images_with_N,total_images,year
0,"[R,G,B]",0,55,2004
1,"[R,G,B]",0,64,2005
2,"[R,G,B]",0,56,2006
3,"[R,G,B]",0,56,2008
4,"[R,G,B,N]",64,64,2009
5,"[R,G,B,N]",56,56,2010
6,"[R,G,B,N]",8,8,2011
7,"[R,G,B,N]",56,56,2012
8,"[R,G,B,N]",8,8,2013
9,"[R,G,B,N]",56,56,2014


In [3]:
## Just for Specific Year Wise Sample

YEAR = 2020  # change to any year from summary_df

start = ee.Date.fromYMD(YEAR, 1, 1)
end   = start.advance(1, 'year')

naip_year = (ee.ImageCollection('USDA/NAIP/DOQQ')
             .filterBounds(meck)
             .filterDate(start, end))

print('Image count:', naip_year.size().getInfo())

first = naip_year.first()
print('Band names of first image:', first.bandNames().getInfo())


NameError: name 'ee' is not defined

### Lets see the imagery on a map (RGB)

In [2]:
m = geemap.Map(center=[35.23, -80.84], zoom=10)

mosaic = naip_year.mosaic().clip(meck)
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255, 'gamma':1.1},
           f'NAIP RGB {YEAR}')
m


NameError: name 'naip_year' is not defined

## Lets dive into the Vegitation Indices [UNSUPERVISED]

### 1) Pick a year and build a clean mosaic 

In [8]:
import ee, geemap
ee.Initialize()

# --- choose a year you like ---
YEAR = 2020  # try 2018/2020/2022 etc.

# Mecklenburg County
meck = (ee.FeatureCollection('TIGER/2018/Counties')
        .filter(ee.Filter.eq('NAME','Mecklenburg'))
        .filter(ee.Filter.eq('STATEFP','37')))

start = ee.Date.fromYMD(YEAR,1,1)
end   = start.advance(1,'year')

naip = (ee.ImageCollection('USDA/NAIP/DOQQ')
        .filterBounds(meck)
        .filterDate(start, end))

print('Tiles this year:', naip.size().getInfo())
first_bands = naip.first().bandNames().getInfo()
print('Band names:', first_bands)

mosaic = naip.mosaic().clip(meck)   # simple mosaic for display/indices


Tiles this year: 57
Band names: ['R', 'G', 'B', 'N']


### 2) If NIR exists → NDVI, SAVI, EVI

In [ ]:
has_nir = 'N' in first_bands

m = geemap.Map(center=[35.23, -80.84], zoom=11)
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255, 'gamma':1.1}, f'NAIP RGB {YEAR}')

if has_nir:
    # NDVI = (N - R) / (N + R)
    ndvi = mosaic.normalizedDifference(['N','R']).rename('NDVI')

    # SAVI = ((N - R) / (N + R + L)) * (1 + L) ; L ~ 0.5 for moderate veg cover
    L = 0.5
    savi = (mosaic.select('N').subtract(mosaic.select('R'))).divide(
            mosaic.select('N').add(mosaic.select('R')).add(L)
          ).multiply(1 + L).rename('SAVI')

    # EVI = 2.5 * (N - R) / (N + 6R - 7.5B + 1)
    evi = (mosaic.select('N').subtract(mosaic.select('R'))).multiply(2.5).divide(
            mosaic.select('N').add(mosaic.select('R').multiply(6))
                  .subtract(mosaic.select('B').multiply(7.5))
                  .add(1)
          ).rename('EVI')

    # Add to map (green palettes)
    green = ['#f7fcf5','#e5f5e0','#c7e9c0','#74c476','#31a354','#006d2c']
    m.addLayer(ndvi, {'min':0, 'max':1, 'palette':green}, 'NDVI')
    m.addLayer(savi, {'min':0, 'max':1.3, 'palette':green}, 'SAVI')
    m.addLayer(evi,  {'min':-1, 'max':1, 'palette':green}, 'EVI')

else:
    print("This year is RGB-only (no NIR). Showing RGB-based indices below.")
m


## SUPERVISED Classification using Random Forest Model

In [67]:
## Build Feature Stack

In [7]:
# Feature stack: NAIP raw bands + NDVI + SAVI
features = mosaic.select(['R','G','B','N']).addBands([ndvi, savi]) #Adding the computed NDVI and SAVI bands as extras.


NameError: name 'mosaic' is not defined

In [69]:
## Code Structure for Training Samples with POIs

In [12]:
# Make a map centered on AOI
m = geemap.Map(center=[35.23, -80.84], zoom=12)
m.add_basemap("SATELLITE") # Add satellite basemap (for visual guidance)
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255}, 'NAIP RGB') # Add your NAIP mosaic (RGB)
m.add_draw_control() # Enable drawing tools
m


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [13]:
#Check number of drawn polygons
print("Polygons drawn so far:", m.user_rois.size().getInfo())

Polygons drawn so far: 0


In [79]:
# Save polygons to GeoJSON Forest
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=r"E:\Others\Erfan Vegetation\POIs\forest_poi.geojson"
)

print("✅ Forest polygons saved")

✅ Forest polygons saved


In [88]:
# Save polygons to GeoJSON Grass
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=r"E:\Others\Erfan Vegetation\POIs\grass_poi.geojson"
)

print("✅ Grass polygons saved")

✅ Grass polygons saved


In [99]:
# Save polygons to GeoJSON Others
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=r"E:\Others\Erfan Vegetation\POIs\others_poi.geojson"
)

print("✅ Others polygons saved")

✅ Others polygons saved


In [ ]:
## Load Training Samples Back

In [9]:
# --- Forest ---
with open(r"E:\Others\Erfan Vegetation\POIs\forest_poi.geojson") as f:
    forest_json = json.load(f)
forest = geemap.geojson_to_ee(forest_json).map(lambda f: f.set("class", 0))

# --- Grass ---
with open(r"E:\Others\Erfan Vegetation\POIs\grass_poi.geojson") as f:
    grass_json = json.load(f)
grass = geemap.geojson_to_ee(grass_json).map(lambda f: f.set("class", 1))

# --- Others (note the filename has 's') ---
with open(r"E:\Others\Erfan Vegetation\POIs\others_poi.geojson") as f:
    others_json = json.load(f)
others = geemap.geojson_to_ee(others_json).map(lambda f: f.set("class", 2))

# Merge all into one FeatureCollection
training_samples = forest.merge(grass).merge(others)

print("Forest points:", forest.size().getInfo())
print("Grass points:", grass.size().getInfo())
print("Others points:", others.size().getInfo())
print("Total training samples:", training_samples.size().getInfo())


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\Others\\Erfan Vegetation\\POIs\\forest_poi.geojson'

In [15]:
# Sample pixel values at training points
training = features.sampleRegions(
    collection=training_samples,
    properties=['class'],
    scale=1  # NAIP resolution = 1 m
)

print("Sampled training data:", training.size().getInfo())


Sampled training data: 390


In [16]:
# Train Random Forest
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training,
    classProperty='class',
    inputProperties=features.bandNames()
)


In [17]:
# Apply classifier
classified = features.classify(classifier)

# Visualization
palette = ['006400', '7CFC00', '808080']  # forest=dark green, grass=light green, others=gray
m.addLayer(classified, {'min': 0, 'max': 2, 'palette': palette}, 'Classification')
m


Map(bottom=414820.0, center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], p…

In [18]:
# Split training data into train/test (70/30 split)
withRandom = training.randomColumn('random')
train_set  = withRandom.filter('random < 0.7')
test_set   = withRandom.filter('random >= 0.7')

# Train on training set
trained = ee.Classifier.smileRandomForest(50).train(
    features=train_set,
    classProperty='class',
    inputProperties=features.bandNames()
)

# Validate on test set
validated = test_set.classify(trained)

conf_matrix = validated.errorMatrix('class', 'classification')
print("Confusion Matrix:", conf_matrix.getInfo())
print("Overall Accuracy:", conf_matrix.accuracy().getInfo())


Confusion Matrix: [[39, 5, 0], [3, 37, 0], [0, 0, 33]]
Overall Accuracy: 0.9316239316239316


In [105]:
# Confusion matrix
conf_matrix = validated.errorMatrix('class', 'classification')

# Overall accuracy
print("Overall Accuracy:", conf_matrix.accuracy().getInfo())

# Kappa coefficient
print("Kappa:", conf_matrix.kappa().getInfo())

# Producer's accuracy (recall)
print("Producer's Accuracy:", conf_matrix.producersAccuracy().getInfo())

# User's accuracy (precision)
print("User's Accuracy:", conf_matrix.consumersAccuracy().getInfo())


Overall Accuracy: 0.9316239316239316
Kappa: 0.8968253968253969
Producer's Accuracy: [[0.8863636363636364], [0.925], [1]]
User's Accuracy: [[0.9285714285714286, 0.8809523809523809, 1]]


## K-FOLD Classification Cross-Validation

In [20]:
# Step 1. Initialize Earth Engine
import ee, geemap, json
ee.Initialize()


In [22]:
# Step 2. Load your POIs from GeoJSON

# --- Forest ---
with open(r"E:\Others\Erfan Vegetation\POIs\forest_poi.geojson") as f:
    forest_json = json.load(f)
forest = geemap.geojson_to_ee(forest_json).map(lambda f: f.set("class", 0))

# --- Grass ---
with open(r"E:\Others\Erfan Vegetation\POIs\grass_poi.geojson") as f:
    grass_json = json.load(f)
grass = geemap.geojson_to_ee(grass_json).map(lambda f: f.set("class", 1))

# --- Other ---
with open(r"E:\Others\Erfan Vegetation\POIs\others_poi.geojson") as f:
    other_json = json.load(f)
other = geemap.geojson_to_ee(other_json).map(lambda f: f.set("class", 2))

# Merge
training_samples = forest.merge(grass).merge(other)
print("Total POIs:", training_samples.size().getInfo())


Total POIs: 390


In [23]:
# Step 3. Build your feature stack (bands + indices)

# Example: your NAIP mosaic
# (Assuming you already made "mosaic" for 2020)
ndvi = mosaic.normalizedDifference(['N','R']).rename('NDVI')
savi = mosaic.expression(
    '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
    {'NIR': mosaic.select('N'), 'RED': mosaic.select('R'), 'L': 0.5}
).rename('SAVI')

# Stack
features = mosaic.select(['R','G','B','N']).addBands([ndvi, savi])


In [24]:
# Step 4. Sample pixel values at POIs
training = features.sampleRegions(
    collection=training_samples,
    properties=['class'],
    scale=1
)

# Clean samples (drop rows missing band values)
bandNames = features.bandNames()
training_clean = training.filter(ee.Filter.notNull(bandNames))

print("Training samples after cleaning:", training_clean.size().getInfo())


Training samples after cleaning: 390


In [26]:
# Step 5. K-fold cross-validation

k = 5
withFolds = training_clean.randomColumn('random')

accuracies = []
conf_matrices = []

for i in range(k):
    fold_min = i / k
    fold_max = (i + 1) / k
    
    # Test fold
    test_fold = withFolds.filter(ee.Filter.gte('random', fold_min)) \
                         .filter(ee.Filter.lt('random', fold_max))
    
    # Train fold = everything else
    train_fold = withFolds.filter(ee.Filter.Or(
        ee.Filter.lt('random', fold_min),
        ee.Filter.gte('random', fold_max)
    ))
    
    # Train classifier
    classifier = ee.Classifier.smileRandomForest(50).train(
        features=train_fold,
        classProperty='class',
        inputProperties=features.bandNames()
    )
    
    # Validate
    validated = test_fold.classify(classifier)
    conf_matrix = validated.errorMatrix('class', 'classification')
    acc = conf_matrix.accuracy()
    
    print(f"Fold {i+1} Confusion Matrix:\n", conf_matrix.getInfo())
    print(f"Fold {i+1} Accuracy:", acc.getInfo(), "\n")
    
    accuracies.append(acc)
    conf_matrices.append(conf_matrix)

# Compute average accuracy
mean_acc = ee.Array(accuracies).reduce(ee.Reducer.mean(), [0])
print("Mean cross-validation accuracy:", mean_acc.getInfo())


Fold 1 Confusion Matrix:
 [[28, 5, 0], [0, 23, 0], [0, 0, 18]]
Fold 1 Accuracy: 0.9324324324324325 

Fold 2 Confusion Matrix:
 [[20, 0, 0], [4, 27, 0], [0, 0, 31]]
Fold 2 Accuracy: 0.9512195121951219 

Fold 3 Confusion Matrix:
 [[24, 1, 0], [2, 26, 2], [0, 0, 28]]
Fold 3 Accuracy: 0.9397590361445783 

Fold 4 Confusion Matrix:
 [[23, 2, 0], [1, 15, 0], [0, 0, 25]]
Fold 4 Accuracy: 0.9545454545454546 

Fold 5 Confusion Matrix:
 [[25, 1, 1], [1, 29, 0], [0, 0, 28]]
Fold 5 Accuracy: 0.9647058823529412 

Mean cross-validation accuracy: [0.9485324635341057]


## Accuracy Assessment using Independent validation data

In [27]:
# Step 1. Open your interactive map

import geemap

m = geemap.Map(center=[35.23, -80.84], zoom=12)
m.add_basemap("SATELLITE")
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255}, 'NAIP RGB')

m


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [33]:
#Check number of drawn polygons
print("Polygons drawn so far:", m.user_rois.size().getInfo())

Polygons drawn so far: 30


In [29]:
# Save polygons to GeoJSON Forest
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=r"E:\Others\Erfan Vegetation\POIs\forest_val_poi.geojson"
)

print("✅ Forest val_poi polygons saved")

✅ Forest val_poi polygons saved


In [31]:
# Save polygons to GeoJSON Grass
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=r"E:\Others\Erfan Vegetation\POIs\grass_val_poi.geojson"
)

print("✅ Grass val_poi polygons saved")

✅ Grass val_poi polygons saved


In [34]:
# Save polygons to GeoJSON Others 
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=r"E:\Others\Erfan Vegetation\POIs\others_val_poi.geojson"
)

print("✅ Others val_poi polygons saved")

✅ Others val_poi polygons saved


In [36]:
# Load your validation POIs


import json, geemap, ee
ee.Initialize()

# --- Forest Validation ---
with open(r"E:\Others\Erfan Vegetation\POIs\forest_val_poi.geojson") as f:
    forest_val_json = json.load(f)
forest_val = geemap.geojson_to_ee(forest_val_json).map(lambda f: f.set("class", 0))

# --- Grass Validation ---
with open(r"E:\Others\Erfan Vegetation\POIs\grass_val_poi.geojson") as f:
    grass_val_json = json.load(f)
grass_val = geemap.geojson_to_ee(grass_val_json).map(lambda f: f.set("class", 1))

# --- Other Validation ---
with open(r"E:\Others\Erfan Vegetation\POIs\others_val_poi.geojson") as f:
    other_val_json = json.load(f)
other_val = geemap.geojson_to_ee(other_val_json).map(lambda f: f.set("class", 2))

# Merge all validation sets
validation_samples = forest_val.merge(grass_val).merge(other_val)
print("Validation samples:", validation_samples.size().getInfo())


Validation samples: 90


In [37]:
validation_data = features.sampleRegions(
    collection=validation_samples,
    properties=['class'],
    scale=1
).filter(ee.Filter.notNull(features.bandNames()))

print("Validation data points:", validation_data.size().getInfo())


Validation data points: 90


In [38]:
# Apply classifier (already trained earlier with training_data)
validated = validation_data.classify(classifier)

# Confusion Matrix
conf_matrix = validated.errorMatrix('class', 'classification')

print("Confusion Matrix:\n", conf_matrix.getInfo())
print("Overall Accuracy:", conf_matrix.accuracy().getInfo())
print("Kappa:", conf_matrix.kappa().getInfo())
print("Producer's Accuracy:", conf_matrix.producersAccuracy().getInfo())
print("User's Accuracy:", conf_matrix.consumersAccuracy().getInfo())


Confusion Matrix:
 [[25, 5, 0], [1, 29, 0], [0, 0, 30]]
Overall Accuracy: 0.9333333333333333
Kappa: 0.9
Producer's Accuracy: [[0.8333333333333334], [0.9666666666666667], [1]]
User's Accuracy: [[0.9615384615384616, 0.8529411764705882, 1]]


### 3)a. Exporting the NDVI TIFF in the Google Drive GEE folder

1 m resolution require around 7-8 GB. So we are coverting our output to 5m.

In [10]:
# Export NDVI or VARI at 5 m resolution to Google Drive

to_export = (ndvi if has_nir else vari).toFloat()

task = ee.batch.Export.image.toDrive(
    image=to_export,
    description=f"meck_{'ndvi' if has_nir else 'vari'}_{YEAR}_5m",
    folder="GEE",  # Google Drive folder name
    fileNamePrefix=f"meck_{'ndvi' if has_nir else 'vari'}_{YEAR}_5m",
    region=meck.geometry(),
    scale=5,       # 5 m resolution
    maxPixels=1e13,
    fileFormat="GeoTIFF"
)

task.start()
print("Started Drive export at 5 m resolution. Check https://drive.google.com -> 'GEE' folder.")


Started Drive export at 5 m resolution. Check https://drive.google.com -> 'GEE' folder.


### 3)b. Exporting the SAVI TIFF in the Google Drive GEE folder

In [11]:
# Export SAVI or VARI at 5 m resolution to Google Drive

to_export = (savi if has_nir else vari).toFloat()

task = ee.batch.Export.image.toDrive(
    image=to_export,
    description=f"meck_{'savi' if has_nir else 'vari'}_{YEAR}_5m",
    folder="GEE",  # Google Drive folder name
    fileNamePrefix=f"meck_{'savi' if has_nir else 'vari'}_{YEAR}_5m",
    region=meck.geometry(),
    scale=5,       # 5 m resolution
    maxPixels=1e13,
    fileFormat="GeoTIFF"
)

task.start()
print("Started Drive export at 5 m resolution. Check https://drive.google.com -> 'GEE' folder.")


Started Drive export at 5 m resolution. Check https://drive.google.com -> 'GEE' folder.


### 4) Check status of the last export task

In [12]:
# Check status of the last export task
print(task.status())


{'state': 'READY', 'description': 'meck_savi_2020_5m', 'priority': 100, 'creation_timestamp_ms': 1758064344038, 'update_timestamp_ms': 1758064344038, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'PAOWQO43T4IGR5OBDJDIHPP3', 'name': 'projects/585047522875/operations/PAOWQO43T4IGR5OBDJDIHPP3'}


**For NDVI and SAVI, the pixel value range is always between −1.0 and +1.0 by definition.**
**Healthy vegetation → 0.2 to 0.8 (forests often higher, grasslands lower)**
**Sparse vegetation / stressed plants → 0.2 to 0.3. Import the Exported TIFF file in ArcGIS Pro you will get teh value. Negative values usually mean water or non-vegetated surfaces.**

## NDVI 3-class thresholding

In [21]:
# --- NDVI 3-class thresholding ---
classified_ndvi = (ndvi.expression(
    "(ndvi < 0.2) ? 0"        # class 0 = others
    ": (ndvi < 0.3) ? 1"      # class 1 = grass
    ": 2",                    # class 2 = trees
    {'ndvi': ndvi})
    .rename('class'))

# --- Add to map ---
palette = ['grey', 'yellow', 'green']  # others, grass, trees
m.addLayer(classified_ndvi, {'min': 0, 'max': 2, 'palette': palette}, 
           'NDVI 3-class (0.2/0.3 thresholds)')

m


Map(bottom=106148821.0, center=[35.195509493079534, -80.98574783228855], controls=(WidgetControl(options=['pos…

In [19]:
# --- SAVI 3-class thresholding ---
classified_savi = (savi.expression(
    "(savi < 0.2) ? 0"        # class 0 = others
    ": (savi < 0.3) ? 1"      # class 1 = grass
    ": 2",                    # class 2 = trees
    {'savi': savi})
    .rename('class'))

# --- Add to map ---
palette = ['grey', 'yellow', 'green']  # others, grass, trees
m.addLayer(classified_savi, {'min': 0, 'max': 2, 'palette': palette}, 
           'SAVI 3-class (0.2/0.3 thresholds)')

m


Map(bottom=26539459.0, center=[35.18661566009878, -80.99460393190385], controls=(WidgetControl(options=['posit…

## Accuracy Assessments (Without manually selecting POIs)

In [14]:
# -------------------------------
# 1) Build a vegetation mask
# -------------------------------
# If you have NIR (most NAIP years): use NDVI threshold.
# If it's RGB-only: use VARI threshold.
if has_nir:
    index_name = 'NDVI'
    index_img  = ndvi  # from your code
    thr        = 0.30  # tune if needed: 0.25–0.35 are common
else:
    # Define VARI if not already defined
    vari = mosaic.expression(
        '(G - R) / (G + R - B + 1e-6)',
        {'R': mosaic.select('R'),
         'G': mosaic.select('G'),
         'B': mosaic.select('B')}
    ).rename('VARI')
    index_name = 'VARI'
    index_img  = vari
    thr        = 0.05   # tune (0.0–0.1 typical)

veg_mask = index_img.gt(thr).rename('vegetation')

# -------------------------------
# 2) Reference dataset: NLCD 2019 -> veg(1)/non-veg(0)
# -------------------------------
nlcd = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select('landcover')

# Consider these as "vegetation": forests, shrub, herbaceous, pasture/hay, crops, wetlands
veg_codes = ee.List([41, 42, 43, 52, 71, 81, 82, 90, 95])
reference = nlcd.remap(
    veg_codes,
    [1,   1,   1,   1,  1,  1,  1,  1,  1],
    0
).rename('veg')  # everything else = 0

# -------------------------------
# 3) Balanced validation points from NLCD (no manual POIs)
# -------------------------------
# Take equal samples from each class to avoid imbalance bias.
validation_points = reference.stratifiedSample(
    numPoints=0,                 # ignored when classPoints provided
    classBand='veg',
    region=meck.geometry(),
    scale=30,                    # NLCD native resolution
    classValues=[0, 1],
    classPoints=[600, 600],      # adjust total as you like
    seed=42,
    geometries=True
)

# -------------------------------
# 4) Align your prediction to NLCD (reduce to 30 m)
# -------------------------------
veg30 = veg_mask.reproject(nlcd.projection()) # lock to NLCD grid

# -------------------------------
# 5) Sample, confusion matrix, and metrics
# -------------------------------
validation = veg30.sampleRegions(
    collection=validation_points,
    properties=['veg'],  # reference label
    scale=30
)

conf_matrix = validation.errorMatrix('veg', 'vegetation')

print(f"Index used: {index_name}, Threshold: {thr}")
print("Confusion Matrix:", conf_matrix.getInfo())
print("Overall Accuracy:", conf_matrix.accuracy().getInfo())
print("Kappa:", conf_matrix.kappa().getInfo())

# Precision (User’s) and Recall (Producer’s) as arrays
precision_ee = conf_matrix.consumersAccuracy()   # per predicted class
recall_ee    = conf_matrix.producersAccuracy()   # per reference class

precision_py = precision_ee.getInfo()            # [[p0, p1]]
recall_py    = recall_ee.getInfo()               # [[r0],[r1]]

# Flatten to Python lists
precisions = precision_py[0]
recalls    = [r[0] for r in recall_py]

# F1 per class + macro-F1
f1_scores = [(2*p*r/(p+r)) if (p+r) > 0 else 0.0 for p, r in zip(precisions, recalls)]
macro_f1  = sum(f1_scores) / len(f1_scores)

print("\nClass-wise metrics:")
print(" Class 0 (non-veg) → Precision: {:.3f}, Recall: {:.3f}, F1: {:.3f}".format(precisions[0], recalls[0], f1_scores[0]))
print(" Class 1 (veg)     → Precision: {:.3f}, Recall: {:.3f}, F1: {:.3f}".format(precisions[1], recalls[1], f1_scores[1]))
print("\nMacro-average F1:", round(macro_f1, 3))


Index used: NDVI, Threshold: 0.3
Confusion Matrix: [[523, 77], [296, 304]]
Overall Accuracy: 0.6891666666666667
Kappa: 0.3783333333333334

Class-wise metrics:
 Class 0 (non-veg) → Precision: 0.639, Recall: 0.872, F1: 0.737
 Class 1 (veg)     → Precision: 0.798, Recall: 0.507, F1: 0.620

Macro-average F1: 0.678


## Accuracy Assessments (Using manually selecting POIs)

In [12]:
import geemap
m = geemap.Map(center=[35.23, -80.84], zoom=12)
m.add_basemap("SATELLITE")   # add high-res satellite basemap
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255}, 'NAIP RGB')
m


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [71]:
# Save the currently drawn points as vegetation = 1
veg_points = ee.FeatureCollection(m.draw_features).map(lambda f: f.set('veg', 1))
print("Vegetation POIs saved:", veg_points.size().getInfo())


Vegetation POIs saved: 0


In [72]:
# Save the new drawn points as non-vegetation = 0
nonveg_points = ee.FeatureCollection(m.draw_features).map(lambda f: f.set('veg', 0))
print("Non-veg POIs saved:", nonveg_points.size().getInfo())


Non-veg POIs saved: 0


In [73]:
validation_points = veg_points.merge(nonveg_points)
print("Total validation POIs:", validation_points.size().getInfo())


Total validation POIs: 0


In [74]:
validation = veg_mask.sampleRegions(
    collection=validation_points,
    properties=['veg'],   # reference label
    scale=5
)

conf_matrix = validation.errorMatrix('veg', 'vegetation')
print("Confusion Matrix:", conf_matrix.getInfo())
print("Overall Accuracy:", conf_matrix.accuracy().getInfo())
print("Kappa:", conf_matrix.kappa().getInfo())
print("Producers Accuracy:", conf_matrix.producersAccuracy().getInfo())
print("Users Accuracy:", conf_matrix.consumersAccuracy().getInfo())


Confusion Matrix: [[0]]
Overall Accuracy: 0
Kappa: 0
Producers Accuracy: [[0]]
Users Accuracy: [[0]]


## Lets dive into the Vegitation Indices [SUPERVISED]

In [24]:
import ee, geemap
ee.Initialize()


In [31]:
import ee, geemap

# Initialize
ee.Initialize()

YEAR = 2020  # change if needed

# Mecklenburg County boundary
meck = (ee.FeatureCollection('TIGER/2018/Counties')
        .filter(ee.Filter.eq('NAME', 'Mecklenburg'))
        .filter(ee.Filter.eq('STATEFP', '37')))

start = ee.Date.fromYMD(YEAR, 1, 1)
end   = start.advance(1, 'year')

# Load NAIP imagery
naip = (ee.ImageCollection('USDA/NAIP/DOQQ')
        .filterBounds(meck)
        .filterDate(start, end))

print('Tiles this year:', naip.size().getInfo())
first_bands = naip.first().bandNames().getInfo()
print('Band names of first image:', first_bands)

# Mosaic to one image
mosaic = naip.mosaic().clip(meck)

# ---- Vegetation Indices ----
has_nir = 'N' in first_bands

# NDVI
if has_nir:
    ndvi = mosaic.normalizedDifference(['N','R']).rename('NDVI')
else:
    ndvi = ee.Image.constant(0).rename('NDVI')

# VARI (RGB-based)
vari = (mosaic.select('G').subtract(mosaic.select('R'))) \
        .divide(mosaic.select('G').add(mosaic.select('R')).subtract(mosaic.select('B'))) \
        .rename('VARI')

# SAVI (Soil Adjusted Vegetation Index), L=0.5
if has_nir:
    savi = (mosaic.select('N').subtract(mosaic.select('R'))) \
           .divide(mosaic.select('N').add(mosaic.select('R')).add(0.5)) \
           .multiply(1.5) \
           .rename('SAVI')
else:
    savi = ee.Image.constant(0).rename('SAVI')

# Stack features
feature_bands = ['R','G','B'] + (['N'] if has_nir else [])
feat_img = mosaic.select(feature_bands).addBands([ndvi, vari, savi]).toFloat()

# ---- Visualization ----
m = geemap.Map(center=[35.23, -80.84], zoom=11)
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255, 'gamma':1.1}, f'NAIP RGB {YEAR}')
if has_nir:
    m.addLayer(ndvi, {'min':0, 'max':1, 'palette':['white','#e5f5e0','#a1d99b','#31a354','#006d2c']}, 'NDVI')
    m.addLayer(savi, {'min':0, 'max':1, 'palette':['white','#f0f9e8','#ccebc5','#7bccc4','#2b8cbe']}, 'SAVI')
m

Tiles this year: 57
Band names of first image: ['R', 'G', 'B', 'N']


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

### Draw and save training polygons (with code)

In [37]:
import ee, geemap, json
ee.Initialize()

# (Your mosaic/NDVI/SAVI code stays as-is)

# show map + drawing tools
m = geemap.Map(center=[35.23, -80.84], zoom=11)
m.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255, 'gamma':1.1}, f'NAIP RGB {YEAR}')
m.add_draw_control()   # <-- this adds polygon/rectangle tools
m


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

### Define the “save” helper (run once)
**This defines a function. It won’t save anything yet.**

**NOW, after running the following code, Draw your FOREST polygons on the map. Click the polygon tool on the map. Draw several polygons covering different forested spots.**

*NB. Important: draw polygons/rectangles, not points.*

In [46]:
import ee, json
ee.Initialize()

def save_drawn_polygons_from_map(map_obj, out_path):
    """
    Save current drawings from geemap.Map to a GeoJSON file,
    keeping only Polygon/MultiPolygon geometries.
    Works when drawings are ee.Feature or plain dicts.
    """
    features = getattr(map_obj, "draw_features", None)
    if not features:
        print("No drawn features found. Draw with the Polygon/Rectangle tool.")
        return

    polys_geojson = []

    # Case A: items are Earth Engine Features
    try:
        if all(hasattr(f, "getInfo") for f in features):
            fc = ee.FeatureCollection(features)
            info = fc.getInfo()  # one client-side call for all
            for f in info.get('features', []):
                geom_type = f.get('geometry', {}).get('type')
                if geom_type in ('Polygon', 'MultiPolygon'):
                    polys_geojson.append(f)
    except Exception as e:
        print("EE conversion attempt message (can be ignored if dict path works):", e)

    # Case B: items are already Python dicts (ipyleaflet/geojson style)
    if not polys_geojson:
        # try treating them as dicts
        for f in features:
            try:
                if isinstance(f, dict):
                    gt = f.get('geometry', {}).get('type')
                    if gt in ('Polygon', 'MultiPolygon'):
                        polys_geojson.append(f)
            except Exception:
                pass

    if not polys_geojson:
        print("No polygons found. Make sure you used the Polygon/Rectangle tool (not points/lines).")
        return

    gj = {"type": "FeatureCollection", "features": polys_geojson}
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(gj, f)
    print(f"Saved {len(polys_geojson)} polygon(s) -> {out_path}")


### Showing the map again to draw the POIs

In [48]:
m.add_draw_control()
m


Map(bottom=207288.0, center=[35.382332329032984, -80.79311370849611], controls=(WidgetControl(options=['positi…

In [49]:
## Run the save function (Forest POIs)
save_drawn_polygons_from_map(m, r"E:\Others\Erfan Vegetation\POIs\forest_training.geojson")


Saved 17 polygon(s) -> E:\Others\Erfan Vegetation\POIs\forest_training.geojson


**Now Clear your drawings and draw POIs for the another class**

In [52]:
## Run the save function (Grass POIs)
save_drawn_polygons_from_map(m, r"E:\Others\Erfan Vegetation\POIs\grass_training.geojson")


Saved 23 polygon(s) -> E:\Others\Erfan Vegetation\POIs\grass_training.geojson


In [ ]:
### Load your training polygons and label them

In [53]:
import geemap, ee
ee.Initialize()

# Update paths if needed
forest_geojson = r"E:\Others\Erfan Vegetation\POIs\forest_training.geojson"
grass_geojson  = r"E:\Others\Erfan Vegetation\POIs\grass_training.geojson"

forest_fc = geemap.geojson_to_ee(forest_geojson).map(lambda f: f.set({'class': 1}))
grass_fc  = geemap.geojson_to_ee(grass_geojson).map(lambda f: f.set({'class': 0}))
training_fc = forest_fc.merge(grass_fc)

print('Forest features:', forest_fc.size().getInfo(),
      'Grass features:',  grass_fc.size().getInfo())


Forest features: 17 Grass features: 23


In [ ]:
### Sample pixels under training polygons

In [54]:
# NAIP is ~1 m; you can use 2 m to sample less densely if it’s huge
sample_scale = 1

training_samples = feat_img.sampleRegions(
    collection=training_fc,
    properties=['class'],
    scale=sample_scale,
    geometries=False
)

print('Training sample count:', training_samples.size().getInfo())


Training sample count: 160778


In [ ]:
### Train/test split + Random Forest

In [55]:
# Add a random column for split
with_random = training_samples.randomColumn('rand', seed=42)
train = with_random.filter(ee.Filter.lt('rand', 0.7))
test  = with_random.filter(ee.Filter.gte('rand', 0.7))

# Input feature names (from your stack)
input_props = feature_bands + ['NDVI', 'VARI', 'SAVI']

classifier = (
    ee.Classifier.smileRandomForest(numberOfTrees=200, seed=42)
      .train(features=train, classProperty='class', inputProperties=input_props)
)

# Classify the full mosaic
classified = feat_img.classify(classifier).rename('class')


In [ ]:
### Accuracy assessment

In [56]:
test_classified = test.classify(classifier)
cm = test_classified.errorMatrix('class', 'classification')
print('Confusion matrix:\n', cm.getInfo())
print('Overall accuracy:', cm.accuracy().getInfo())
print('Kappa:', cm.kappa().getInfo())


Confusion matrix:
 [[1486, 207], [129, 46105]]
Overall accuracy: 0.9929893379514678
Kappa: 0.8947995169346902


In [ ]:
### Visualize

In [57]:
palette = ['#a1d99b', '#006d2c']  # [grass=0, forest=1]
m2 = geemap.Map(center=[35.23, -80.84], zoom=11)
m2.addLayer(mosaic, {'bands':['R','G','B'], 'min':0, 'max':255, 'gamma':1.1}, f'NAIP RGB {YEAR}')
m2.addLayer(classified, {'min':0, 'max':1, 'palette':palette}, 'Grass(0) vs Forest(1)')
m2


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…